<a href="https://colab.research.google.com/github/c-labropoulos/NN-for-lithological-segmentation/blob/main/UNETTEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
!pip install rasterio
import os
import numpy as np
import pandas as pd
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix as sk_confusion_matrix
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate
import torch.optim as optim
from torchvision import transforms
from sklearn.model_selection import train_test_split
import glob
from PIL import Image
import numpy as np
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 56.9 MB/s eta 0:00:00


In [3]:
# Import the PyTorch library
import torch

# Check for GPU availability
# If a GPU with CUDA support is available, the code will use it for computation.
# Otherwise, it will use the CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print the device being used (either "cuda" for GPU or "cpu" for CPU)
# This will help users to verify if their GPU and CUDA installations are recognized
# and if the code is running on the GPU as expected.
print("Using device:", device)

Using device: cuda


In the above cell :

1.	We import the torch library, which is the main PyTorch library for tensor computations and neural network building.
2.	We check if a GPU with CUDA support is available on the system using torch.cuda.is_available(). If a GPU is available, the code will use it for computation; otherwise, it will use the CPU.
3.	We create a device variable that holds the device type that will be used for computation (either "cuda" for GPU or "cpu" for CPU).
4.	We print the device type being used to help users verify if their GPU and CUDA installations are recognized and if the code is running on the GPU as expected.


In [4]:
def read_raster_image(file_path):
    """
    Read a raster image from the provided file path and return the first band's data as a numpy array.

    Args:
        file_path (str): The path of the raster image file to be read.

    Returns:
        numpy.ndarray: A numpy array containing the data of the first band of the raster image.
    """
    # Use the rasterio library to open the raster image file
    with rasterio.open(file_path) as src:
        # Read the first band of the image using the `read` method
        image_data = src.read(1)

    # Return the data of the first band as a numpy array
    return image_data

In this code snippet:
1.	We define a read_raster_image function that takes a file path as an argument.
2.	Using the rasterio.open context manager, we open the raster image file at the provided file path.
3.	We read the first band of the image using the read method of the rasterio src object and store the data in the image_data variable.
4.	After the context manager closes the file, we return the data of the first band as a numpy array.






In [5]:
def get_file_paths(folder_path, file_extension=".tif"):
    """
    Traverse a given folder and its subfolders to find files with the specified file extension,
    and return a list of their file paths.

    Args:
        folder_path (str): The path of the folder to search for files.
        file_extension (str, optional): The file extension to search for. Defaults to ".tif".

    Returns:
        list: A list of file paths for the files with the specified extension found in the folder and its subfolders.
    """
    # Initialize an empty list to store the file paths
    file_paths = []

    # Use os.walk to traverse the folder and its subfolders
    for root, _, files in os.walk(folder_path):
        # Iterate through the files in the current folder
        for file in files:
            # Check if the file has the specified extension
            if file.endswith(file_extension):
                # Append the file path to the list
                file_paths.append(os.path.join(root, file))

    # Return the list of file paths
    return file_paths



In this code snippet:
1.	We define a get_file_paths function that takes a folder path and an optional file extension as arguments.
2.	We initialize an empty list file_paths to store the paths of the files we find.
3.	We use the os.walk function to traverse the folder and its subfolders, iterating through the files in each folder.
4.	For each file, we check if it has the specified file extension. If it does, we append its path to the file_paths list.
5.	After traversing the entire folder structure, we return the list of file paths.


In [6]:
# Read class mapping from CSV
def read_class_mapping(file_path):
    df = pd.read_csv(file_path)
    class_mapping = {row['pixel_value']: row['class_name'] for _, row in df.iterrows()}
    return class_mapping

In [7]:
def iou_metric(y_true, y_pred, class_mapping):
    iou_scores = []
    
    for class_pixel_value in class_mapping:
        if class_mapping[class_pixel_value] == 'nan':
            continue

        y_true_class = y_true == class_pixel_value
        y_pred_class = y_pred == class_pixel_value

        intersection = np.logical_and(y_true_class, y_pred_class)
        union = np.logical_or(y_true_class, y_pred_class)

        if np.sum(union) == 0:
            continue

        iou_score = np.sum(intersection) / np.sum(union)
        iou_scores.append(iou_score)

    return np.mean(iou_scores)


In [8]:
def pixel_accuracy(y_true, y_pred):
    correct_pixels = np.sum(y_true == y_pred)
    total_pixels = y_true.size
    return correct_pixels / total_pixels

from sklearn.metrics import precision_score, recall_score, f1_score


In [9]:

def compute_metrics(y_true, y_pred, num_classes):
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()
    
    precision = precision_score(y_true_flat, y_pred_flat, labels=np.arange(num_classes), average='weighted')
    recall = recall_score(y_true_flat, y_pred_flat, labels=np.arange(num_classes), average='weighted')
    f1 = f1_score(y_true_flat, y_pred_flat, labels=np.arange(num_classes), average='weighted')
    
    return precision, recall, f1

from sklearn.metrics import confusion_matrix



In [10]:
def compute_confusion_matrix(y_true, y_pred, num_classes):
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()
    
    cm = sk_confusion_matrix(y_true_flat, y_pred_flat, labels=np.arange(num_classes))
    return cm


In [11]:

class RasterDataset(Dataset):
    def __init__(self, root_dir, patch_size, stride, train_ratio, is_train=True):
        """
        Initialize the RasterDataset class.

        Args:
            root_dir (str): The root directory containing the raster images.
            patch_size (int): The size of the patches to be extracted from the images.
            stride (int): The step size used when extracting patches from the images.
            train_ratio (float): The ratio of images to be used for training.
            is_train (bool, optional): A flag indicating whether the dataset is for training or testing. Defaults to True.
        """
        self.root_dir = root_dir
        self.patch_size = patch_size
        self.stride = stride
        self.train_ratio = train_ratio
        self.is_train = is_train

        self.image_paths = self.get_image_paths(root_dir)

        num_train = int(len(self.image_paths) * train_ratio)
        if is_train:
            self.image_paths = self.image_paths[:num_train]
        else:
            self.image_paths = self.image_paths[num_train:]

        self.image_patches = self.create_patches()

    def get_image_paths(self, root_dir):
        """
        Get the sorted file paths for all .tif images in the specified directory.

        Args:
            root_dir (str): The directory containing the raster images.

        Returns:
            list: A list of sorted file paths for the .tif images in the directory.
        """
        image_folder = os.path.join(root_dir)
        image_paths = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.tif')])
        return image_paths

    def create_patches(self):
        """
        Extract patches from the images in the dataset.

        Returns:
            list: A list of image patches.
        """
        image_patches = []

        for image_path in self.image_paths:
            image = Image.open(image_path)
            width, height = image.size

            for y in range(0, height - self.patch_size + 1, self.stride):
                for x in range(0, width - self.patch_size + 1, self.stride):
                    image_patch = image.crop((x, y, x + self.patch_size, y + self.patch_size))
                    image_patches.append(image_patch)

        return image_patches

    def preprocess(self, image):
        """
        Preprocess an image patch, converting it to a PyTorch tensor and normalizing it.

        Args:
            image (PIL.Image): The image patch to be preprocessed.

        Returns:
            torch.Tensor: The preprocessed image patch as a PyTorch tensor.
        """
        # Convert the PIL Image to a numpy array
        image = np.array(image)

        # Normalize the image to the range [0, 1]
        image = image / 255.0

        # Convert the numpy array to a PyTorch tensor and add a channel dimension
        image = torch.from_numpy(image).float().unsqueeze(0)

        return image

    def __len__(self):
        """
        Get the number of image patches in the dataset.

        Returns:
            int: The number of image patches.
        """
        return len(self.image_patches)

    def __getitem__(self, index):
        """
        Get an image patch and its corresponding mask from the dataset.

        Args:
            index (int): The index of the image patch in the dataset.

        Returns:
            tuple: A tuple containing the image patch and its corresponding mask as PyTorch tensors.
        """
        image_patch = self.image_patches[index]
        image_patch = self.preprocess(image_patch)
        mask = (image_patch != 0).long() # Convert the mask tensor to Long
        mask = mask.squeeze(0) # Remove the channel dimension
        return image_patch, mask


In [12]:
# Create dataset and data loader
folder_path = "/content/drive/MyDrive/raster_to_be_used"

# Set the parameters for creating the dataset
patch_size = 128  # Size of the patches extracted from the images
stride = 32  # Stride used when extracting patches from the images
train_ratio = 0.8  # Ratio of the dataset to be used for training

# Create the training dataset using the RasterDataset class
train_dataset = RasterDataset(folder_path, patch_size, stride, train_ratio, is_train=True)
# Create the DataLoader for the training dataset, with a batch size of 4, shuffling the data, and using 2 workers for parallel processing
train_data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)

# Create the testing dataset using the RasterDataset class
test_dataset = RasterDataset(folder_path, patch_size, stride, train_ratio, is_train=False)
# Create the DataLoader for the testing dataset, with a batch size of 4, without shuffling the data, and using 2 workers for parallel processing
test_data_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)


In the above code snippet, we create the datasets and data loaders for both training and testing. The parameters for patch size, stride, and train ratio are set. The DataLoader objects are created with specific batch sizes, shuffling options, and numbers of parallel processing workers.

In [13]:
class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(UNet, self).__init__()

        # Contracting path
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)

        self.pool = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)

        # Expanding path
        self.upconv4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = self.conv_block(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = self.conv_block(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = self.conv_block(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = self.conv_block(128, 64)

        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))

        bottleneck = self.bottleneck(self.pool(enc4))

        upconv4 = self.upconv4(bottleneck)
        dec4 = self.dec4(torch.cat((enc4, upconv4), dim=1))
        upconv3 = self.upconv3(dec4)
        dec3 = self.dec3(torch.cat((enc3, upconv3), dim=1))
        upconv2 = self.upconv2(dec3)
        dec2 = self.dec2(torch.cat((enc2, upconv2), dim=1))
        upconv1 = self.upconv1(dec2)
        dec1 = self.dec1(torch.cat((enc1, upconv1), dim=1))

        return self.final_conv(dec1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )




In the code snippet above, a UNet class is defined, which inherits from PyTorch's nn.Module. The U-Net architecture consists of an encoder (contracting path), a bottleneck, and a decoder (expanding path). The U-Net is designed for semantic segmentation tasks.
The __init__ method of the UNet class defines the layers in the contracting path, the bottleneck, and the expanding path.
The forward method is responsible for defining the forward pass of the network. It takes an input tensor x, processes it through the contracting path, bottleneck, and expanding path, and returns the final output.
The conv_block method is a helper function for defining a sequence of convolutional and ReLU layers. It takes the number of input channels and output channels and returns a sequential container with the two convolutional layers followed by ReLU activations. This block is used in both the contracting and expanding paths of the U-Net architecture.


In [14]:
# Customize in_channels and out_channels according to your data
in_channels = 1  # Number of input channels (1 for single-band images)
out_channels = 18  # Number of output channels (number of classes for multi-class segmentation)

model = UNet(in_channels=in_channels, out_channels=out_channels)
# Move the model to the GPU
model = model.to(device)


In [15]:

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set the number of training epochs
num_epochs = 10

# Move the model to the GPU if available
model = model.to(device)

# Iterate over each epoch
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print("-" * 10)

    # Set the model to training mode
    model.train()
    train_loss = 0.0
    # Iterate over the training data loader
    for batch_idx, (images, masks) in enumerate(train_data_loader):
        # Move images and masks to the GPU if available
        images, masks = images.to(device), masks.to(device)

        # Reset the optimizer's gradients
        optimizer.zero_grad()
        
        # Perform forward pass with the model
        outputs = model(images)
        # Compute the loss
        loss = criterion(outputs, masks)
        # Perform backpropagation
        loss.backward()
        # Update the model's weights
        optimizer.step()
        
        # Accumulate the training loss
        train_loss += loss.item()
        
    # Calculate the average training loss
    train_loss /= len(train_data_loader)
    print(f"Train Loss: {train_loss:.4f}")

    # Set the model to evaluation mode
    model.eval()
    eval_loss = 0.0
    # Evaluate the model without gradient updates
    with torch.no_grad():
        # Iterate over the test data loader
        for batch_idx, (images, masks) in enumerate(test_data_loader):
            # Move images and masks to the GPU if available
            images, masks = images.to(device), masks.to(device)

            # Perform forward pass with the model
            outputs = model(images)
            # Compute the loss
            loss = criterion(outputs, masks)
            
            # Accumulate the evaluation loss
            eval_loss += loss.item()
    
    # Calculate the average evaluation loss
    eval_loss /= len(test_dataset)
    print(f"Evaluation loss: {eval_loss:.4f}")

Epoch 1/10
----------
Train Loss: 603.3939
Evaluation loss: 0.0055
Epoch 2/10
----------
Train Loss: 0.0103
Evaluation loss: 0.0006
Epoch 3/10
----------
Train Loss: 0.0068
Evaluation loss: 0.0004
Epoch 4/10
----------
Train Loss: 0.0052
Evaluation loss: 0.0003
Epoch 5/10
----------
Train Loss: 0.0044
Evaluation loss: 0.0002
Epoch 6/10
----------
Train Loss: 0.0040
Evaluation loss: 0.0001
Epoch 7/10
----------
Train Loss: 810.6022
Evaluation loss: 0.0013
Epoch 8/10
----------
Train Loss: 0.0068
Evaluation loss: 0.0009
Epoch 9/10
----------
Train Loss: 0.0053
Evaluation loss: 0.0005
Epoch 10/10
----------
Train Loss: 0.0034
Evaluation loss: 0.0003


In [16]:

class_mapping = read_class_mapping('/content/drive/MyDrive/class_mapping.csv')
num_classes = len(class_mapping)

iou_scores = []
precision_scores = []  # Add this line
recall_scores = []     # Add this line
f1_scores = []         # Add this line
mean_confusion_matrices = []

model.eval()
with torch.no_grad():
    for images, masks in test_data_loader:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        masks_np = masks.cpu().numpy()
        preds_np = preds.cpu().numpy()
        
        iou_score = iou_metric(masks_np, preds_np, class_mapping)
        iou_scores.append(iou_score)
        
        precision = precision_score(masks_np.flatten(), preds_np.flatten(), labels=np.arange(num_classes), average='weighted', zero_division=0)
        precision_scores.append(precision)
        
        recall = recall_score(masks_np.flatten(), preds_np.flatten(), labels=np.arange(num_classes), average='weighted', zero_division=0)
        recall_scores.append(recall)
        
        f1 = f1_score(masks_np.flatten(), preds_np.flatten(), labels=np.arange(num_classes), average='weighted', zero_division=0)
        f1_scores.append(f1)
        
        cm = compute_confusion_matrix(masks_np, preds_np, num_classes)
        mean_confusion_matrices.append(cm)

mean_iou = np.mean(iou_scores)
print(f"Mean IoU score on test dataset: {mean_iou:.4f}")

mean_precision = np.mean(precision_scores)
print(f"Mean precision on test dataset: {mean_precision:.4f}")

mean_recall = np.mean(recall_scores)
print(f"Mean recall on test dataset: {mean_recall:.4f}")

mean_f1 = np.mean(f1_scores)
print(f"Mean F1-score on test dataset: {mean_f1:.4f}")

mean_cm = np.mean(mean_confusion_matrices, axis=0)
print(f"Mean confusion matrix on test dataset:\n {mean_cm}")

Mean IoU score on test dataset: 0.8920
Mean precision on test dataset: 0.9998
Mean recall on test dataset: 0.9998
Mean F1-score on test dataset: 0.9998
Mean confusion matrix on test dataset:
 [[2.40848262e+03 1.38793456e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.22392638e+00 6.30621564e+04 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+0

In [18]:

import gc


tf.keras.backend.clear_session()



# Trigger garbage collection
gc.collect()

111